In [1]:
import os
from openai import OpenAI
import sys
sys.path.append('../..')
import utils
import openai



client=OpenAI(api_key=""

In [2]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message.content

In [3]:
user_input= "Tell me about the SmartX ProPhone and Televisions and Home Theater Systems"
user="u r shit"
delimiter = "```"

In [4]:
#Step 1: Check input to see if it flags the Moderation API or is a prompt injection
def check_input_with_moderation(user_input):
    response = client.moderations.create(input=user_input)
    moderation_output = response.results[0]

    if moderation_output.flagged:
        print("Step 1: Input flagged by Moderation API.")
        return "Sorry, we cannot process this request."
    else:
        print("Step 1: Input passed moderation check.")
        return "Proceed to next step"


In [5]:
check_input_with_moderation(user_input)
category_and_product_response = utils.find_category_and_product_only(user_input, utils.get_products_and_category())

Step 1: Input passed moderation check.


In [6]:
category_and_product_response# am doing this to test if my bot is able to classify the catefories and the products successfully
# before doing any specific explanation 

"```python\n[\n    {'category': 'Smartphones and Accessories'},\n    {'category': 'Televisions and Home Theater Systems', 'products': ['SmartX ProPhone']}\n]\n```"

In [11]:
 # Step 2: Extract the list of products

# so the category_and_product_response this i got above is not a json string but a code block which i hv to clean first 
# i hv to convert this to a list this is called parsing
check_input_with_moderation(user_input)
category_and_product_response = utils.find_category_and_product_only(user_input, utils.get_products_and_category())

Step 1: Input passed moderation check.


In [12]:
category_and_product_list = utils.read_string_to_list(category_and_product_response)
category_and_product_list# so now i got a list of dictionaries

[{'category': 'Smartphones and Accessories'},
 {'category': 'Televisions and Home Theater Systems'}]

In [13]:
 # step 3: to create data based on customer enquiry from company data base
product_information=utils.generate_output_string(category_and_product_list)
#product_information

In [14]:
# Above i got a json string i did this step to collect the actual data from my company data base about the product or category enquired
# by the customer then i will feed this info th the gpt so that it wll return a response to the user based on the info above
# and npot hallucinate or create it own data thats not prsent in the company database.

In [15]:
# step 4: now i use the product info to make by bot create a good answer to my client so i guide
# my bot to not hallucinate or give unreal info
system_message = f"""
    You are a customer service assistant for a large electronic store. \
    Respond in a friendly and helpful tone, with concise answers. \
    Make sure to ask the user relevant follow-up questions.
    """
messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': f"{delimiter}{user_input}{delimiter}"},
        {'role': 'assistant', 'content': f"Relevant product information:\n{product_information}"}
]

all_messages=[]
final_response = get_completion_from_messages(all_messages + messages)
all_messages = all_messages + messages[1:]# this means jus include 1st and 2nd message cos syst.messge is the same throuhg out


In [16]:
final_response # so here the bot gave a good clear reply based on the product info i fed to the assistant role

'The SmartX ProPhone is a powerful smartphone with a 6.1-inch display, 128GB storage, 12MP dual camera, and 5G capability. It is priced at $899.99. We also have accessories like the MobiTech PowerCase for $59.99, which is a protective case with a built-in battery compatible with the SmartX ProPhone.\n\nFor Televisions and Home Theater Systems, we offer the CineView 4K TV for $599.99, featuring a 55-inch display, 4K resolution, HDR, and smart TV capabilities. Additionally, the SoundMax Home Theater system is available for $399.99, providing a 5.1 channel setup with a wireless subwoofer and Bluetooth connectivity.\n\nDo you have any specific questions about these products or are you looking for something specific within these categories?'

In [17]:
#all_messages

In [18]:
# Step 5: Put the answer through the Moderation API
response = client.moderations.create(input=final_response)
moderation_output = response.results[0]

if moderation_output.flagged: 
        print("Step 5: Response flagged by Moderation API.")
        print("Sorry, we cannot provide this information.")

else:
        print("Step 5: Response passed moderation check.")

Step 5: Response passed moderation check.


In [19]:
# Step 6: Ask the model if the response answers the initial user query well
user_message = f"""
    Customer message: {delimiter}{user_input}{delimiter}
    Agent response: {delimiter}{final_response}{delimiter}

    Does the response sufficiently answer the question?
    say "yes" if so otherwise "let me connect u to a human"
    """
messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
]
evaluation_response = get_completion_from_messages(messages)

In [20]:
evaluation_response#i am asking the assistant to evaluate its respons wrt the user message and check if it gv a proper reply

'yes'

In [21]:
# step 7: if the ecvaluation of the final response is correct we  write a function to send the message to the user otherwise we con
#connecct to a humann

if evaluation_response=="yes":
    print(final_response,all_messages)
else:
    print("I'm unable to provide the information you're looking for. I'll connect you with a human representative for further assistance.",all_messages)

The SmartX ProPhone is a powerful smartphone with a 6.1-inch display, 128GB storage, 12MP dual camera, and 5G capability. It is priced at $899.99. We also have accessories like the MobiTech PowerCase for $59.99, which is a protective case with a built-in battery compatible with the SmartX ProPhone.

For Televisions and Home Theater Systems, we offer the CineView 4K TV for $599.99, featuring a 55-inch display, 4K resolution, HDR, and smart TV capabilities. Additionally, the SoundMax Home Theater system is available for $399.99, providing a 5.1 channel setup with a wireless subwoofer and Bluetooth connectivity.

Do you have any specific questions about these products or are you looking for something specific within these categories? [{'role': 'user', 'content': '```Tell me about the SmartX ProPhone and Televisions and Home Theater Systems```'}, {'role': 'assistant', 'content': 'Relevant product information:\n{\n    "name": "SmartX ProPhone",\n    "category": "Smartphones and Accessories"

In [24]:
pip install panel

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/29.5 MB ? eta -:--:--
   ---- ----------------------------------- 3.7/29.5 MB 27.3 MB/s eta 0:00:01
   -------------- ------------------------- 10.5/29.5 MB 31.2 MB/s eta 0:00:01
   ------------------------- -------------- 18.6/29.5 MB 33.6 MB/s eta 0:00:01
   ------------------------------------- -- 27.5/29.5 MB 36.4 MB/s eta 0:00:01
   ---------------------------------------- 29.5/29.5 MB 35.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------- 7.0/7.0 MB 39.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ------------------------- -------------- 8.1/12.6 MB 38.7 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 36.0 MB/s eta 0:00:00
   ---------------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [25]:
import panel as pn
pn.extension()

In [41]:
inp = pn.widgets.TextInput(placeholder='wats ur name')# inp is a text widget which creates the text box where the 
#user types the qtn
button_conversation = pn.widgets.Button(name="Ask Assistant")# this triggers the assistant 

user_input = inp.value_input
if inp.value =='':
    return
    inp.value = ''
    global context
response,context=process_user_input(user_input,context)
context.append({'role':'assistant', 'content':f"{response}"})# context already has user msg and product info hence we r append
# only the response to it to create a memory and to pass as the next context 

panels.append(
    pn.Row('User:', pn.pane.Markdown(user_input, width=600)))# here panels creates a row names user with user message
panels.append(
    pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))# here panels creates a row named assistant
# with responsse and color to differentiate both. so panels creat a visual chat boxes.

SyntaxError: cannot assign to attribute here. Maybe you meant '==' instead of '='? (1241005316.py, line 6)

In [42]:
def collect_messages(event=None, debug=False):
    user_input = inp.value_input
    if inp.value == '':
        return
        inp.value = ''
        global context
    response,context=process_user_input(user_input,context)
    context.append({'role':'assistant', 'content':f"{response}"})# context already has user msg and product info hence we r append
# only the response to it to create a memory and to pass as the next context 

    panels.append(
       pn.Row('User:', pn.pane.Markdown(user_input, width=600)))# here panels creates a row names user with user message
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))# here panels creates a row named assistant
# with responsse and color to differentiate both. so panels creat a visual chat boxes.
# one for user and one for assistant response, wen the buttonis clicked again another 2 is created and so on
    return pn.Column(*panels)

In [44]:
panels = []
context = [ {'role':'system', 'content':"You are Service Assistant"} ]  
interactive_conversation = pn.bind(collect_messages, button_conversation)# so eachtime the button is clicked the collect_messages 
#function runs
dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] TextInput(placeholder='wats ur name')
    [1] Row
        [0] Button(name='Ask Assistant')
    [2] ParamFunction(function, _pane=Str, defer_load=False, height=300, loading_indicator=True)